#### 4.4 뷰티풀 수프로 일별 시세 읽어오기

#### 뷰티풀 수프 공식문서
#### https://www.crummy.com/software/BeautifulSoup/bs4/doc/#beautiful-soup-documentation
##### 네이버 금융 웹스크래핑 차단 (urllib.request.urlopen() or pandas.read_html() > http 패킷 헤더의 웹브라우저정보(User-Agent)를 체크함) > requests 라이브러리 이용해야 웹브라우저정보를 넘겨줘야함

##### 4.4.3 맨 뒤 페이지 수 가져오기

In [1]:
from bs4 import BeautifulSoup as bs

In [13]:
import requests

In [31]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=068270'

In [32]:
with urlopen(url) as doc:
    html = bs(requests.get(url, headers={'User-agent' : 'Mozilla/5.0'}).text, 'lxml')
    pgrr = html.find("td", class_='pgRR')
    print(pgrr.a['href'])

/item/sise_day.nhn?code=068270&page=388


In [33]:
print(pgrr.prettify())

<td class="pgRR">
 <a href="/item/sise_day.nhn?code=068270&amp;page=388">
  맨뒤
  <img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
</td>



In [22]:
print(pgrr.text)


맨뒤
				




In [26]:
with urlopen(url) as doc:
    html = bs(requests.get(url, headers={'User-agent' : 'Mozilla/5.0'}).text, 'lxml')
    pgrr = html.find("td", class_='pgRR')
    s = str(pgrr.a['href']).split('=')
    last_page = s[-1]
    print(last_page)

388


##### 4.4.4 전체 페이지의 데이터 읽어오기 > 일별시세는 테이블형태 > read_html()로 읽어와서 데이터프레임화하기

In [29]:
import pandas as pd
df = pd.DataFrame()

In [39]:
for page in range(1, int(last_page)+1):
    pg_url = '{}&page={}'.format(url,page)
    df = df.append(pd.read_html(requests.get(pg_url, headers = {'User-agent' : 'Mozilla/5.0'}).text)[0])

In [49]:
df = df.dropna()
print(df)
type(df)

          date     close    diff      open      high       low    volume
1   2021.03.19  287000.0  5000.0  288000.0  290500.0  284000.0  452626.0
2   2021.03.18  292000.0  1500.0  295500.0  299500.0  292000.0  401776.0
3   2021.03.17  293500.0  3000.0  296500.0  298000.0  291000.0  360539.0
4   2021.03.16  296500.0  7500.0  290500.0  298000.0  289000.0  410799.0
5   2021.03.15  289000.0  8000.0  296500.0  297000.0  289000.0  474511.0
..         ...       ...     ...       ...       ...       ...       ...
1   2005.07.25    5650.0    70.0    5500.0    5950.0    5500.0   61036.0
2   2005.07.22    5580.0   160.0    5850.0    5850.0    5530.0   69921.0
3   2005.07.21    5740.0   810.0    6450.0    6580.0    5730.0  182685.0
4   2005.07.20    6550.0  1150.0    7690.0    7690.0    6550.0  422688.0
5   2005.07.19    7700.0  2500.0    6700.0    7700.0    6510.0  499088.0

[11625 rows x 7 columns]


pandas.core.frame.DataFrame

In [41]:
df = df.rename(columns= {'날짜' : 'date', '종가' : 'close', '전일비' : 'diff', '시가' : 'open', '고가' : 'high', '저가' : 'low', '거래량' : 'volume'})

In [42]:
df.head(10)

,date,close,diff,open,high,low,volume
1,2021.03.19,287000.0,5000.0,288000.0,290500.0,284000.0,452626.0
2,2021.03.18,292000.0,1500.0,295500.0,299500.0,292000.0,401776.0
3,2021.03.17,293500.0,3000.0,296500.0,298000.0,291000.0,360539.0
4,2021.03.16,296500.0,7500.0,290500.0,298000.0,289000.0,410799.0
5,2021.03.15,289000.0,8000.0,296500.0,297000.0,289000.0,474511.0
9,2021.03.12,297000.0,4000.0,296000.0,301000.0,293000.0,416941.0
10,2021.03.11,293000.0,7500.0,285500.0,296500.0,284500.0,660641.0
11,2021.03.10,285500.0,2000.0,291500.0,296500.0,284000.0,494901.0
12,2021.03.09,287500.0,3500.0,288000.0,292000.0,280500.0,853291.0
13,2021.03.08,291000.0,10500.0,304000.0,304000.0,290500.0,595881.0
